In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [79]:
data = pd.read_csv('TATN1day1.csv', sep = ',')
#data.head(1)
#data.shape
data

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,TATN,D,20170103,0,426.45,447.85,424.55,445.20,1272240
1,TATN,D,20170104,0,444.85,446.40,434.10,446.40,1195220
2,TATN,D,20170105,0,445.60,448.65,418.25,424.25,2121160
3,TATN,D,20170106,0,420.75,426.80,415.50,423.45,801450
4,TATN,D,20170109,0,425.15,426.45,418.20,425.80,889110
...,...,...,...,...,...,...,...,...,...
955,TATN,D,20201016,0,432.40,433.80,426.20,429.20,4213273
956,TATN,D,20201019,0,430.70,432.80,414.90,415.70,3847080
957,TATN,D,20201020,0,415.70,424.80,406.40,421.20,7552688
958,TATN,D,20201021,0,421.70,428.60,416.10,416.90,5154493


In [80]:
#Избавляемся от ненужных столбцов
data.drop('<OPEN>', axis = 1, inplace = True)
data.drop('<HIGH>', axis = 1, inplace = True)
data.drop('<LOW>', axis = 1, inplace = True)
data.drop('<VOL>', axis = 1, inplace = True)
data.drop('<PER>', axis = 1, inplace = True)
data.drop('<TICKER>', axis = 1, inplace = True)
data.drop('<TIME>', axis = 1, inplace = True)
#Добавляем нужные нам столбцы
data['long'] = 0
data['short'] = 0
data['long exit'] = 0
data['short exit'] = 0
data['position'] = 0
data[''] = 0
data[''] = data[''].astype('float64') #Этот стобец требуется нам в типе float
data['Сделка'] = ''
data['Дневная доходность'] = 0
data['Дневная доходность'] = data['Дневная доходность'].astype('float64')
#data['<DATE>'] = pd.to_datetime(data['<DATE>']) Так же не лишним будет изменить тип данных столбца <DATE>
data

,<DATE>,<CLOSE>,long,short,long exit,short exit,position,,Сделка,Дневная доходность
0,20170103,445.20,0,0,0,0,0,0.0,,0.0
1,20170104,446.40,0,0,0,0,0,0.0,,0.0
2,20170105,424.25,0,0,0,0,0,0.0,,0.0
3,20170106,423.45,0,0,0,0,0,0.0,,0.0
4,20170109,425.80,0,0,0,0,0,0.0,,0.0
...,...,...,...,...,...,...,...,...,...,...
955,20201016,429.20,0,0,0,0,0,0.0,,0.0
956,20201019,415.70,0,0,0,0,0,0.0,,0.0
957,20201020,421.20,0,0,0,0,0,0.0,,0.0
958,20201021,416.90,0,0,0,0,0,0.0,,0.0


In [81]:
data.dtypes

<DATE>                  int64
<CLOSE>               float64
long                    int64
short                   int64
long exit               int64
short exit              int64
position                int64
                      float64
Сделка                 object
Дневная доходность    float64
dtype: object

In [82]:
#Заносим информацию в столбцах в numpy массивы
close = np.array(data['<CLOSE>'])
long = np.array(data['long'])
short = np.array(data['short'])
long_exit = np.array(data['long exit'])
short_exit = np.array(data['short exit'])
position = np.array(data['position'])
pustoi_stolbets =np.array(data[''])
sdelka =np.array(data['Сделка'])
dnevnaiadohodnost =np.array(data['Дневная доходность'])
pribiln = 0

In [83]:
#Сам алгоритм
for i in range(2,len(close)):
    #long
    if close[i-2] < close[i-1] < close[i] and position[i-1] == 0:
        long[i] = 1
    else:
        long[i] = 0
    #short
    if close[i-2] > close[i-1] > close[i] and position[i-1] == 0:
        short[i] = 1
    else:
        short[i] = 0
    #long exit
    if close[i-2] > close[i-1] > close[i] and position[i-1] == 1:
        long_exit[i] = 1
    else:
        long_exit[i] = 0
    #short exit
    if close[i-2] < close[i-1] < close[i] and position[i-1] == -1:
        short_exit[i] = 1
    else:
        short_exit[i] = 0
    #position
    if position[i-1] == 0:
        position[i] = long[i] - short[i]
    elif long_exit[i] + short_exit[i] == 1:
        position[i] = 0
    else:
        position[i] = position[i-1]
    #Pustoi stolb
    if position[i-1] == 0 and position[i] == 0:
        pustoi_stolbets[i] = 0
    elif position[i-1] == position[i]:
        pustoi_stolbets[i] = pustoi_stolbets[i-1]
    else:
        pustoi_stolbets[i] = close[i]
    #Сделка
    if position[i-1] != 0 and position[i-1] != position[i]:
        sdelka[i] = (pustoi_stolbets[i]/pustoi_stolbets[i-1] - 1) * position[i-1]
    if position[i-1] != 0:
        dnevnaiadohodnost[i] = (close[i] / close[i-1] - 1) * position[i-1]
    else:
        dnevnaiadohodnost[i] = 0
       
        
data['long'] = long
data['short'] = short
data['long exit'] = long_exit
data['short exit'] = short_exit 
data['position'] = position
data[''] = pustoi_stolbets
data['Сделка'] = sdelka * 100
data['Дневная доходность'] = dnevnaiadohodnost *100
data.head(20)

,<DATE>,<CLOSE>,long,short,long exit,short exit,position,,Сделка,Дневная доходность
0,20170103,445.20,0,0,0,0,0,0.00,,0.000000
1,20170104,446.40,0,0,0,0,0,0.00,,0.000000
2,20170105,424.25,0,0,0,0,0,0.00,,0.000000
3,20170106,423.45,0,1,0,0,-1,423.45,,0.000000
4,20170109,425.80,0,0,0,0,-1,423.45,,-0.554965
5,20170110,428.60,0,0,0,1,0,428.60,-1.2162,-0.657586
6,20170111,431.25,1,0,0,0,1,431.25,,0.000000
7,20170112,438.00,0,0,0,0,1,431.25,,1.565217
8,20170113,433.10,0,0,0,0,1,431.25,,-1.118721
9,20170116,427.25,0,0,1,0,0,427.25,-0.927536,-1.350727


In [97]:
data2 = pd.DataFrame()
dohodnost = {'Name':'Доходность с реинвест'}
srarifm = {'Name':'Среднеарифметическая доходность'}
srgeom = {'Name':'Среднегеометрическая доходность'}
kolvo_sdeloc = {'Name':'Количество сделок'}
pros_pribiln = {'Name':'% приб.сделок'}
pribiln = {'Name':'Приб.сделок'}
otr = {'Name':'Отр.сделок'}
zero = {'Name':'Нулевых сделок'}
kolvo_dney = {'Name':'Количество дней'}
data2 = data2.append(dohodnost, ignore_index = True)
data2 = data2.append(srarifm, ignore_index = True)
data2 = data2.append(srgeom, ignore_index = True)
data2 = data2.append(kolvo_sdeloc, ignore_index = True)
data2 = data2.append(pros_pribiln, ignore_index = True)
data2 = data2.append(pribiln, ignore_index = True)
data2 = data2.append(otr, ignore_index = True)
data2 = data2.append(kolvo_dney, ignore_index = True)
data2

,Name
0,Доходность с реинвест
1,Среднеарифметическая доходность
2,Среднегеометрическая доходность
3,Количество сделок
4,% приб.сделок
5,Приб.сделок
6,Отр.сделок
7,Количество дней
